In [1]:
import numpy as np

# Parameters
S0 = 110.0     # Initial stock price
K = 100.0      # Strike price
r = 0.01       # Risk-free rate
sigma = 0.3    # Volatility
T = 1.0        # Time to maturity (1 year)
m = 12         # Number of monitoring points (monthly)
dt = T / m

<font size=5, font color='blue'> Here, we use the same set of random number sequences (10000000 simulations) to approximate the value of all Greeks.

In [3]:
n_sims = 10000000

n_sims_half = n_sims // 2

Z = np.random.randn(n_sims_half, m)
    
# Moment matching: force sample mean=0 and std=1
Z = (Z - Z.mean()) / Z.std()
    
# Create antithetic variates
Z_antithetic = -Z
    
Z_all = np.vstack((Z, Z_antithetic))

<font size=5, font color='blue'> Define a function to compute the analytical solution of the price of the Asian option based on geometric average stock price, which will be used as a control variate.

In [20]:
from scipy.stats import norm
def compute_geom_asian_price(S0, K, r, sigma, T, m):
    t = np.arange(1, m+1) * (T / m)
    T_bar = t.mean()
    
    sum_ = 0.0
    for i in range(1, m+1):
        sum_ += (2*i - 1) * t[m - i]
    sigma_bar_sq = (sigma**2 / (m**2 * T_bar)) * sum_
    
    dividend = 0.5*sigma**2-0.5*sigma_bar_sq
    d = (np.log(S0/K)+(r-dividend+0.5*sigma_bar_sq)*T_bar)/np.sqrt(sigma_bar_sq*T_bar)
    
    price_geo = np.exp(-dividend*T_bar-r*(T-T_bar))*S0*norm.cdf(d)-np.exp(-r*T)*K*norm.cdf(d-np.sqrt(sigma_bar_sq*T_bar))
    
    return price_geo

In [32]:
original_geo_price = compute_geom_asian_price(S0, K, r, sigma, T, m)
original_geo_price

13.40575400389794

In [8]:
def compute_asian_price_with_control(n_sims, S0, K, r, sigma, T, m, C_geom_exact):
    """
    Parameters:
      n_sims       : total number of simulation paths
      S0, K, r, sigma, T, m: standard option parameters
      C_geom_exact : known closed-form price of the geometric Asian call

    Returns:
      price_est : control-variate adjusted option price estimate
    """
    increments = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z_all
    log_paths = np.cumsum(increments, axis=1)
    S_paths = S0 * np.exp(log_paths)  

    S_arith = np.mean(S_paths, axis=1)
    S_geom  = np.exp(np.mean(np.log(S_paths), axis=1))

    payoff_arith = np.maximum(S_arith - K, 0)
    payoff_geom  = np.maximum(S_geom  - K, 0)

    # --- Discounted payoffs ---
    discount = np.exp(-r * T)
    disc_arith = discount * payoff_arith
    disc_geom  = discount * payoff_geom

    # --- Control variate method ---
    # b* = cov(X, Y) / var(X)
    # Y = disc_arith, X = disc_geom, E[X] = C_geom_exact
    # Y_cv = Y - b*( X - E[X] )
    X = disc_geom
    Y = disc_arith
    X_mean = X.mean()
    Y_mean = Y.mean()

    cov_XY = np.mean((X - X_mean) * (Y - Y_mean))  
    var_X  = np.mean((X - X_mean)**2)            

    b_est = cov_XY / var_X
    Y_cv = Y - b_est * (X - C_geom_exact)

    price_est = np.mean(Y_cv)

    return price_est

<font size=5, font color='blue'> Use a range of shock values with respect to the stock price S0 to estimate the value of Delta and Gamma

In [50]:
S0_shock_list = [S0*i for i in [0.001,0.005,0.01,0.05,0.1]]
S0_shock_list

[0.11, 0.55, 1.1, 5.5, 11.0]

<font size=5, font color='blue'> Delta is about 0.73

In [51]:
for S0_shock in S0_shock_list:
    new_geo_price_up = compute_geom_asian_price(S0+S0_shock, K, r, sigma, T, m)
    new_geo_price_down = compute_geom_asian_price(S0-S0_shock, K, r, sigma, T, m)

    delta = (compute_asian_price_with_control(10000000, S0+S0_shock, K, r, sigma, T, m, new_geo_price_up)-
    compute_asian_price_with_control(10000000, S0-S0_shock, K, r, sigma, T, m, new_geo_price_down))/(2*S0_shock)

    print('delta='+str(delta))

delta=0.7317495931443606
delta=0.7317524306387778
delta=0.7316792125435627
delta=0.7286424796600692
delta=0.7194214481323106


<font size=5, font color='blue'> Gamma is about 0.016

In [52]:
for S0_shock in S0_shock_list:
    new_geo_price_up1 = compute_geom_asian_price(S0+S0_shock, K, r, sigma, T, m)
    new_geo_price_down1 = compute_geom_asian_price(S0-S0_shock, K, r, sigma, T, m)

    gamma = (compute_asian_price_with_control(10000000, S0+S0_shock, K, r, sigma, T, m, new_geo_price_up1)
    -2*compute_asian_price_with_control(10000000, S0, K, r, sigma, T, m, original_geo_price)
    + compute_asian_price_with_control(10000000, S0-S0_shock, K, r, sigma, T, m, new_geo_price_down1))/(S0_shock**2)

    print('gamma='+str(gamma))

gamma=0.01639043340162516
gamma=0.016415970260000735
gamma=0.01636625728684525
gamma=0.016304449969714065
gamma=0.016215681220902074


<font size=5, font color='blue'> Vega is about 22

In [54]:
new_geo_price_vol_up = compute_geom_asian_price(S0, K, r, sigma+0.01, T, m)
new_geo_price_vol_down = compute_geom_asian_price(S0, K, r, sigma-0.01, T, m)

vega = (compute_asian_price_with_control(10000000, S0, K, r, sigma+0.01, T, m, new_geo_price_vol_up)-
    compute_asian_price_with_control(10000000, S0, K, r, sigma-0.01, T, m, new_geo_price_vol_down))/(2*0.01)

print('vega='+str(vega))

vega=21.589758280208393


<font size=5, font color='blue'> Use a range of shock values with respect to the interest rate r to estimate the value of Rho

In [56]:
rate_shock_list = [0.001,0.0025,0.005,0.0075,0.01]
rate_shock_list

[0.001, 0.0025, 0.005, 0.0075, 0.01]

<font size=5, font color='blue'> Rho is about 30

In [57]:
for r_shock in rate_shock_list:
    new_geo_price_rate_up = compute_geom_asian_price(S0, K, r+r_shock, sigma, T, m)
    new_geo_price_rate_down = compute_geom_asian_price(S0, K, r-r_shock, sigma, T, m)

    rho = (compute_asian_price_with_control(10000000, S0, K, r+r_shock, sigma, T, m, new_geo_price_rate_up)-
        compute_asian_price_with_control(10000000, S0, K, r-r_shock, sigma, T, m, new_geo_price_rate_down))/(2*r_shock)

    print('rho='+str(rho))

rho=30.437190081688215
rho=30.437170296105975
rho=30.43724469040452
rho=30.437310839916403
rho=30.436843555078053
